In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
# Import the data into a Pandas DataFrame
ward_df = pd.read_csv("resources/Census_stats_consolidated_ward.csv")
ward_df.head()

,Ward,Owned_2016,Rental_2016,hh_with_children_2016,incomeaverage2015,incomemedian2015,Owned_2011,Rental_2011,hh_with_children_2011,incomeaverage2010,incomemedian2010,yoy_incomemedian_change
0,1,9920,8385,7965,70939,60352,10195,7955,8730,66001,55027,9.7
1,2,11700,7765,8500,80685,64543,11460,7620,8495,72100,57718,11.8
2,3,14280,5780,7925,110440,81839,14220,5390,8790,95259,72175,13.4
3,4,12630,9155,8565,141783,78617,12665,8445,9000,106513,69501,13.1
4,5,19050,11415,10985,126802,80076,17920,9380,9775,110919,75035,6.7


In [3]:
crime2014_df = pd.read_csv("resources/MCI_2014.csv")
crime2014_df.head()

,reportedyear,crime,Neighbourhood,Lat,Long
0,2014,Robbery,North Riverdale (68),43.668449,-79.343094
1,2014,Break and Enter,Black Creek (24),43.759285,-79.507927
2,2014,Assault,Brookhaven-Amesbury (30),43.697552,-79.501663
3,2014,Assault,Rexdale-Kipling (4),43.721703,-79.571510
4,2014,Assault,Lambton Baby Point (114),43.663891,-79.503487


In [4]:
crime2018_df = pd.read_csv("resources/MCI_2018.csv")
crime2018_df.head()

,reportedyear,crime,Neighbourhood,Lat,Long
0,2018,Assault,Agincourt North (129),43.796127,-79.261513
1,2018,Assault,Agincourt North (129),43.796127,-79.261513
2,2018,Assault,Agincourt North (129),43.797062,-79.283440
3,2018,Assault,Agincourt North (129),43.797409,-79.281662
4,2018,Assault,Agincourt North (129),43.797409,-79.281662


In [5]:
engine = create_engine(f'sqlite:///to_ward_project.sqlite')

In [6]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [7]:
class Wards(Base):
    __tablename__ = 'Wards'
    id = Column(Integer, primary_key=True)
    Ward = Column(Integer)
    Owned_2016 = Column(Integer)
    Rental_2016 = Column(Integer)
    hh_with_children_2016 = Column(Integer)
    incomeaverage2015 = Column(Integer)
    incomemedian2015 = Column(Integer)
    Owned_2011 = Column(Integer)
    Rental_2011 = Column(Integer)
    hh_with_children_2011 = Column(Integer)
    incomeaverage2010 = Column(Integer)
    incomemedian2010 = Column(Integer)
    yoy_incomemedian_change = Column(Integer)

In [8]:
class Crime2014(Base):
    __tablename__ = 'Crime2014'
    id = Column(Integer, primary_key=True)
    reportedyear = Column(Integer)
    crime = Column(String(255))
    Neighbourhood = Column(String(255))
    Lat = Column(Integer)
    Long = Column(Integer)

In [9]:
class Crime2018(Base):
    __tablename__ = 'Crime2018'
    id = Column(Integer, primary_key=True)
    reportedyear = Column(Integer)
    crime = Column(String(255))
    Neighbourhood = Column(String(255))
    Lat = Column(Integer)
    Long = Column(Integer)

In [10]:
conn = engine.connect()
Base.metadata.create_all(conn) 

In [11]:
for index, item in ward_df.iterrows():
    ward = Wards(
        Ward=item['Ward'],
        Owned_2016=item['Owned_2016'],
        Rental_2016=item['Rental_2016'],
        hh_with_children_2016=item['hh_with_children_2016'],
        incomeaverage2015=item['incomeaverage2015'],
        incomemedian2015=item['incomemedian2015'],
        Owned_2011=item['Owned_2011'],
        Rental_2011=item['Rental_2011'],
        hh_with_children_2011=item['hh_with_children_2011'],
        incomeaverage2010=item['incomeaverage2010'],
        incomemedian2010=item['incomemedian2010'],
        yoy_incomemedian_change=item['yoy_incomemedian_change']
    )
    session.add(ward)
    session.commit()

In [12]:
for index, item in crime2014_df.iterrows():
    crime2014 = Crime2014(
        reportedyear=item['reportedyear'],
        crime=item['crime'],
        Neighbourhood=item['Neighbourhood'],
        Lat=item['Lat'],
        Long=item['Long']
    )
    session.add(crime2014)
    session.commit()

In [13]:
for index, item in crime2018_df.iterrows():
    crime2018 = Crime2018(
        reportedyear=item['reportedyear'],
        crime=item['crime'],
        Neighbourhood=item['Neighbourhood'],
        Lat=item['Lat'],
        Long=item['Long']
    )
    session.add(crime2018)
    session.commit()